In [4]:
from pyspark.sql.functions import *

#### Transformando os arquivos de log em um DataFrame

In [5]:
arq_jul = sqlContext.read.text("wasb://nomedocluster@j4wqp6rby7ga2.blob.core.windows.net/HdiNotebooks/PySpark/access_log_Jul95")

df_Jul95 = arq_jul.select(regexp_extract('value', r'^([^\s]+\s)', 1).alias('HOST'),
                          regexp_extract('value', r'^.*\[(\d\d/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]', 1).alias('TIMESTAMP'),
                          regexp_extract('value', r'^.*"\w+\s+([^\s]+)\s+HTTP.*"', 1).alias('PATH'),
                          regexp_extract('value', r'^.*"\s+([^\s]+)', 1).cast('integer').alias('STATUS'),
                          regexp_extract('value', r'^.*\s+(\d+)$', 1).cast('integer').alias('SIZE'))



In [6]:
arq_aug = sqlContext.read.text("wasb://nomedocluster@j4wqp6rby7ga2.blob.core.windows.net/HdiNotebooks/PySpark/access_log_Aug95")

df_Aug95 = arq_aug.select(regexp_extract('value', r'^([^\s]+\s)', 1).alias('HOST'),
                          regexp_extract('value', r'^.*\[(\d\d/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]', 1).alias('TIMESTAMP'),
                          regexp_extract('value', r'^.*"\w+\s+([^\s]+)\s+HTTP.*"', 1).alias('PATH'),
                          regexp_extract('value', r'^.*"\s+([^\s]+)', 1).cast('integer').alias('STATUS'),
                          regexp_extract('value', r'^.*\s+(\d+)$', 1).cast('integer').alias('SIZE'))


#### 1. Número de hosts únicos.

In [8]:
host_Jul = df_Jul95.select(countDistinct("HOST")
                           .alias("Número de hosts únicos(JULHO)")).show()

+-----------------------------+
|Número de hosts únicos(JULHO)|
+-----------------------------+
|                        81983|
+-----------------------------+

In [9]:
host_Aug = df_Aug95.select(countDistinct("HOST")
                           .alias("Número de hosts únicos(AGOSTO)")).show()

+------------------------------+
|Número de hosts únicos(AGOSTO)|
+------------------------------+
|                         75060|
+------------------------------+

#### 2. O total de erros 404.

In [10]:
df_Jul95.select('STATUS')\
        .where('STATUS == 404').count()

10845

In [11]:
df_Aug95.select('STATUS')\
        .where('STATUS == 404').count()

10056

#### 3. Os 5 URLs que mais causaram erro 404.

In [12]:
# Seleciona coluna Host,com Status seja igual a 404.
# Coloca o número 1 em toda a linha que tiver "404", criando uma coluna chamada "count".
# Agrupa todo host igual.
# Faz uma soma da coluna por agrupamento.
# Ordena a coluna COUNT de forma decrescente 

df_Jul95.select(["HOST"])\
        .where('STATUS == 404')\
        .withColumn('COUNT',lit(1))\
        .groupBy('HOST')\
        .sum('COUNT')\
        .orderBy(desc('sum(COUNT)')).show(5)

+--------------------+----------+
|                HOST|sum(COUNT)|
+--------------------+----------+
|hoohoo.ncsa.uiuc....|       251|
|jbiagioni.npt.nuw...|       131|
|piweba3y.prodigy....|       110|
|piweba1y.prodigy....|        92|
|phaelon.ksc.nasa....|        64|
+--------------------+----------+
only showing top 5 rows

In [13]:
df_Aug95.select(["HOST"])\
        .where('STATUS == 404')\
        .withColumn('count',lit(1))\
        .groupBy('Host')\
        .sum('count')\
        .orderBy(desc('sum(count)')).show(5)

+--------------------+----------+
|                Host|sum(count)|
+--------------------+----------+
|dialip-217.den.mm...|        62|
|piweba3y.prodigy....|        47|
|       155.148.25.4 |        44|
|       maz3.maz.net |        39|
|      gate.barr.com |        38|
+--------------------+----------+
only showing top 5 rows